In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# gdrive_home_dir = '/content/drive/My Drive/'


# !ls '{gdrive_home_dir}'







In [ ]:
# !pip install flickrapi

# import flickrapi

In [ ]:
# Photo Source URL description: https://www.flickr.com/services/api/misc.urls.html

# Find the id of a group by group name: https://www.flickr.com/services/api/explore/flickr.urls.lookupGroup
# Get photos from a group: https://www.flickr.com/services/api/explore/flickr.groups.pools.getPhotos

# flickr photo search api: https://www.flickr.com/services/api/explore/flickr.photos.search

# Flickr photo licenses explained: https://www.flickr.com/creativecommons/
# Flick photo licenses: https://www.flickr.com/services/api/explore/flickr.photos.licenses.getInfo

# sample download scripts: https://gist.github.com/zmwangx/b1c16b197b5416143c7a
#                          https://www.programcreek.com/python/example/6468/flickrapi.FlickrAPI
#% reset

In [1]:
'''
https://stackoverflow.com/questions/1994037/flickr-api-returning-duplicate-photos 

It is possible to retrieve more than 4000 images from flickr; your query has to be paginated by (for example) temporal range 
such that the total number of images from that query is not more than 4000. You can also use other parameters such as bounding 
box to limit the total number of images in the response.

For example, if you are searching with the tag 'dogs', this is what you can do ( binary search over time range):

Specify a minimum date and a maximum date in the request url, such as Jan 1st, 1990 and Jan 1st 2015.
Inspect the total number of images in the response. If it is more than 4000, then divide the temporal range into two and 
work on the first half until you get less than 4000 images from the query. Once you get that, request all the pages from 
that time range, and move on to the next interval and do the same until (a) Number of required images is met (b) searched
all over the initial time interval.

'''

#%reload_ext downloader_wrapper

from flickr_dloader_utils import *
import time
import datetime


# Recommended way for creating a multi-process program in python
if __name__ == '__main__':

    
    # number of images to download
    num_images_to_download = 50000

    # Specify whether to download from a group or by tags
    mode = 'tags' # possible values: 'group' or 'tags'

        # name of the group (if downloading from a group, ignored otherwise)
    group_name = 'portrait-gallery'
    group_name = 'selfie-shots'

    # tags to be used to searching (if downloading using search tags, ignored otherwise)
    tags = 'wefie'

    base_download_dir = '/disks/data/datasets/selfie_project_datasets/downloaded_images/flickr'
    
    # to try time intervals 
    SECONDS_IN_A_YEAR = 31536000    
    
    CURRENT_TIME = int(time.time())
    
    # maximum number of times to retry if an image info page results in error
    MAX_RETRY_COUNT = 3

    # maximum number of images to assign to an image downloader process
    MAX_LOAD_PER_IMG_DLOADER = 200 
    
    # maximum number of images information to be contained in a query result page.
    # NOTE: Flickr sends the same set of image information if used a number higher than 100.
    MAX_IMG_PER_PAGE = 100    
    
    # hack since api downloads 100 less than the specified number of images
    num_images_to_download +=100 
    
    # finds the number of pages needed to be downloaded
    #num_pages = int(math.ceil(num_images_to_download/MAX_IMG_PER_PAGE))

    #print(f'Number of pages needed for {num_images_to_download} images: {num_pages}')
    
    # base download directory
    #download_dir = f'{gdrive_home_dir}/dataset/test_dataset'
        
            
    if mode == 'tags':        
        download_dir = f'{base_download_dir}/{tags}_tags'
    else:
        download_dir = f'{base_download_dir}/{group_name}_group'
        
    
    # settings for downloading images from a group or by search keywords/tags
    configs = { 'group': {
                            'search_criteria' : group_name,
                            'download_dir' : download_dir,    
                            #'image_info_save_file' : 'query_results_bygroup__' + group_name,
                            'url_downloader' : download_flickr_img_by_group, # image info downloader function for groups
                        },
               'tags': {
                            'search_criteria' : tags,
                            'download_dir' : download_dir,    
                            #'image_info_save_file' : 'query_results_bytags__' + ".".join(tags.split(",")),
                            'url_downloader' : download_flickr_img_url_by_tag, # image info downloader function for tags
                        }
              }

    
    if mode == 'tags':
        
        retry_count = 0
        is_page_error = True
        # find the number of images available for these tags
        while retry_count < MAX_RETRY_COUNT and is_page_error:  
            try:  

                result = configs[mode]['url_downloader'](configs[mode]['search_criteria'])
                is_page_error = False 
            except Exception as e:
                # enforced delay not to overwhelm the Flickr API  
                retry_count += 1
                print('Retry attempt: ', retry_count)
                time.sleep(2)    


        # Number of pages in the result    
        pagecount = result['photos']['pages']
        imgCount = int(result['photos']['total'])

        print(f"Total available images: {imgCount}")
        print(f'Total available pages: {pagecount}')

        results = []
        search_time_intervals = []

        # split the date range so that each query results in less than total 4000 images; otherwise flickr returns duplicates
        if(imgCount > 4000):
            
            print('More than 4k images in query; segmenting the query')
            photo_uploaddates = [photo['dateupload'] for photo in result['photos']['photo'] ]
            photo_uploaddates.sort()
            start_upload_date = int(photo_uploaddates[0])
            #end_upload_date = start_upload_date + SECONDS_IN_A_YEAR

            #print(first_photo_upload_date)                  
            #print(result)

            #unique_query_photos = { photo['id']:photo for result in results for photo in result['photos']['photo']}
            #unique_query_photos = {}
            total_img_count = 0

            while( total_img_count < num_images_to_download and start_upload_date < CURRENT_TIME ):
                interval_length = (CURRENT_TIME - start_upload_date)//2  #10*SECONDS_IN_A_YEAR
                end_upload_date = start_upload_date + interval_length

                start_time_h = datetime.datetime.utcfromtimestamp(start_upload_date).strftime('%Y-%m-%d %H:%M:%SZ')
                end_time_h = datetime.datetime.utcfromtimestamp(end_upload_date).strftime('%Y-%m-%d %H:%M:%SZ')


                retry_count = 0
                is_page_error = True
                while retry_count < MAX_RETRY_COUNT and is_page_error:  
                    try:  
                        # execute the query for finding out the number of pages available 
                        result = configs[mode]['url_downloader'](configs[mode]['search_criteria'], pagenum = None, 
                                                                   show_log = False, start_date = start_upload_date, end_date = end_upload_date)

                        is_page_error = False 
                    except Exception as e:

                        # enforced delay not to overwhelm the Flickr API  
                        retry_count += 1
                        print('Retry attempt: ', retry_count)
                        time.sleep(1)    

                time.sleep(0.5)
                imgCount = int(result['photos']['total'])
                print("image count: ", imgCount, " (between ", end="") 
                print("start: ", start_time_h, " end:", end_time_h, ")" )

                while(imgCount > 4000 and interval_length > 1):

                    interval_length //= 2
                    end_upload_date = start_upload_date + interval_length

                    start_time_h = datetime.datetime.utcfromtimestamp(start_upload_date).strftime('%Y-%m-%d %H:%M:%SZ')
                    end_time_h = datetime.datetime.utcfromtimestamp(end_upload_date).strftime('%Y-%m-%d %H:%M:%SZ')

                    retry_count = 0
                    is_page_error = True
                    while retry_count < MAX_RETRY_COUNT and is_page_error:  
                        try:  
                            # execute the query for finding out the number of pages available 
                            result = configs[mode]['url_downloader'](configs[mode]['search_criteria'], pagenum = None, 
                                                                                                   show_log = False, 
                                        start_date = start_upload_date, end_date = end_upload_date)

                            is_page_error = False 
                        except Exception as e:
                            # enforced delay not to overwhelm the Flickr API  
                            retry_count += 1
                            print('Retry attempt: ', retry_count)
                            time.sleep(2)                

                    time.sleep(0.5)
                    imgCount = int(result['photos']['total'])
                    print("Search... image count: ", imgCount, " (between ", end="") 
                    print("start: ", start_time_h, " end:", end_time_h, ")" )

                # found query with less than 4k images, so add the start and end times to the list 
                search_time_intervals.append( (start_upload_date, end_upload_date ) )
                total_img_count += imgCount

                # update start time for the next query
                start_upload_date = end_upload_date + 1

                print("Total image count is: ", total_img_count)
                print()

        else:
            results.append(result)
            photo_uploaddates = [photo['dateupload'] for photo in result['photos']['photo'] ]
            photo_uploaddates.sort()
            start_upload_date = int(photo_uploaddates[0])
            end_upload_date = CURRENT_TIME
            search_time_intervals.append( (start_upload_date, end_upload_date ) )
            
        print(search_time_intervals)

        query_photos = {}

        for start_upload_date, end_upload_date in search_time_intervals:
            retry_count = 0
            is_page_error = True
            while retry_count < MAX_RETRY_COUNT and is_page_error:  
                try:  
                    # execute the query for finding out the number of pages available 
                    # execute the query for finding out the number of pages available 
                    result = configs[mode]['url_downloader'](configs[mode]['search_criteria'], pagenum = None, 
                                                                                               show_log = False, 
                                    start_date = start_upload_date, end_date = end_upload_date)
                    is_page_error = False 
                except Exception as e:
                    # enforced delay not to overwhelm the Flickr API  
                    retry_count += 1
                    print('Retry attempt: ', retry_count)
                    time.sleep(2)    


                # Number of pages in the result    
                pagecount = result['photos']['pages']

                page_idx = 0


                has_stalled = False

                image_counts = []  
                # download image info (can't be done parallely since can't instantiate more than one Flickr api instances with 
                # the same api key: possibly can be done by using one key per instance)          
                while len(query_photos.items()) < num_images_to_download and page_idx < pagecount and not has_stalled:

                    page_idx += 1

                    is_page_error = True

                    retry_count = 0

                    while retry_count < MAX_RETRY_COUNT and is_page_error:  
                        try:  
                            # download image info as a dictionary and append to a list  
                            result = configs[mode]['url_downloader'](configs[mode]['search_criteria'], page_idx, 
                                                                               show_log = True, 
                                    start_date = start_upload_date, end_date = end_upload_date)

                            is_page_error = False 
                        except Exception as e:
                            # enforced delay not to overwhelm the Flickr API  
                            retry_count += 1
                            print('Retry attempt: ', retry_count)
                            time.sleep(2)

                    results.append(result)
                    # enforced delay not to overwhelm the Flickr API  
                    time.sleep(0.5)

                    # construct a dictionary of returned photos to eliminate ducplicate images (based on image id)      
                    query_photos_this_page = { photo['id']:photo for photo in result['photos']['photo']}

                    query_photos.update(query_photos_this_page)

                    # compute the number of image information obtained       
                    actual_total_image_count = len(query_photos.items()) # Flickr returns 100 less images for some reason

                    image_counts.append(actual_total_image_count)

                    print(f'Obtained total {len(query_photos_this_page.items())} image information from {page_idx} pages, total obtained:{actual_total_image_count}')

                    if(len(image_counts) > 5 and image_counts[-1] < image_counts[-5] + 20):
                        has_stalled = True

        print(f'Attempting to download total {actual_total_image_count} images')          


        ## Download the images from the downloaded image information           

        # update num_process to reflect actual image count
        num_processes = int(math.ceil(actual_total_image_count/MAX_LOAD_PER_IMG_DLOADER))

        print(f'Number of image downloaders: {num_processes}')

        # arrange the image downloader arguments for multi-process environment          
        args = get_worker_args(num_processes, query_photos, MAX_LOAD_PER_IMG_DLOADER, configs[mode]['download_dir'], 
                                'c', None)

        # for suppressing process output
        def mute():
            sys.stdout = open(os.devnull, 'w')   

        start = time.time()

        with Pool(num_processes) as p:
            p.map(downloader_wrapper, args);

        end = time.time()

        print(f'Elapsed time {end-start} seconds')
    
    else:   # for downloading from groups; date range not allowed in API call
        
        # finds the number of pages needed to be downloaded
        num_pages = int(math.ceil(num_images_to_download/MAX_IMG_PER_PAGE))

    
        # execute the query for finding out the number of pages available 
        result = configs[mode]['url_downloader'](configs[mode]['search_criteria'])

        # Number of pages in the result    
        pagecount = result['photos']['pages']

        print(f"Total available images: {result['photos']['total']}")
        print(f'Total available pages: {pagecount}')

        results = []

        # download image info (can't be done parallely since can't instantiate more than one Flickr api instances with 
        # the same api key)
        for i in range( min(num_pages, pagecount) ):
            
            # download image info as a dictionary and append to a list  
            results.append(configs[mode]['url_downloader'](configs[mode]['search_criteria'], i+1, show_log = True))

            # enforced delay not to overwhelm the Flickr API  
            time.sleep(0.5)

        # construct a dictionary of returned photos to eliminate ducplicate images (based on image id)      
        query_photos = { photo['id']:photo for result in results for photo in result['photos']['photo']}

        # compute the number of image information obtained       
        actual_total_image_count = len(query_photos.items()) # Flickr returns 100 less images for some reason

        print(f'Obtained total {actual_total_image_count} image information') 
              
              
        print(f'Attempting to download total {actual_total_image_count} images')          
              
        # update num_process to reflect actual image count
        num_processes = int(math.ceil(actual_total_image_count/MAX_LOAD_PER_IMG_DLOADER))

        print(f'Number of image downloaders: {num_processes}')
              
        # arrange the image downloader arguments for multi-process environment          
        args = get_worker_args(num_processes, query_photos, MAX_LOAD_PER_IMG_DLOADER, 
                               configs[mode]['download_dir'], 'c', None)

        # for suppressing process output
        def mute():
            sys.stdout = open(os.devnull, 'w')   

        start = time.time()

        with Pool(num_processes) as p:
            p.map(downloader_wrapper, args);

        end = time.time()

        print(f'Elapsed time {end-start} seconds')

Total available images: 103
Total available pages: 2
[(1390240042, 1558409890)]
Obtained total 100 image information from 1 pages, total obtained:100
Obtained total 1 image information from 2 pages, total obtained:101
Attempting to download total 101 images
Number of image downloaders: 1
Elapsed time 17.756003856658936 seconds
